# Wilfire Impact Estimate in Pueblo, CO
## Notebook 1: Calculating Fire Distances

## Initial Data Exploration and Distance Computation
In this analysis, I will develop an estimate of smoke intensity from wildfires in Pueblo, CO. I will then assess how predictive the smoke intensity score I hypothesize is of the Air Quality Index (AQI), a metric commonly used by the US Environmental Protection Agency.

To estimate smoke intensity from wildfire, I'll need as much data as possible about wildfires that occurred close to Pueblo, CO. In this notebook, I import that data and extract the distance from each wildfire in the dataset to Pueblo, CO.

The data I am importing here is from the [Combined wildland fire datasets for the United States and certain territories, 1800s-Present](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) dataset, provided by the US Geographic Survey. 

### ATTRIBUTION
The code and comments below are adopted, with light modifications, from Dr. David McDonald,
who provided them for use in DATA 512, a course in the University of Washington MS of Data
Science Program. The code is provided and utilized here under the [Creative Commons CC-BY license.](https://creativecommons.org/licenses/by/4.0/)

## Environment Setup

First, I import required packages do some basic environmental setup:

In [1]:
import os
import json
from pyproj import Transformer, Geod
from tqdm import tqdm

PROJECT_DIR = "/Users/daniel_personal/Documents/uw/msds_courses/data512/final_project"

os.chdir(PROJECT_DIR) # make it easier to find the .env file and the Reader object


# Reader provided by Dr. David McDonald -- see attribution above
from wildfire.Reader import Reader

DATA_NAME = "raw_data/USGS_Wildland_Fire_Combined_Dataset.json"

DATA_FILEPATH = os.path.join(PROJECT_DIR, DATA_NAME)

wfreader = Reader(DATA_FILEPATH)

## Dataset Import

I begin by importing the USGS Wildfires dataset as a JSON file. This includes wildfires and prescribed burns – I filter out the latter for most of my analyses since the assignment spec indicates we are interested in *wildfires*.

In [3]:
#
#    This sample code will load the whole sample (extracted data) file, or a small amount of the complete dataset.
#
MAX_FEATURE_LOAD = 1e20 # a very large number – no practical limit
feature_list = list()
feature_count = 0
# A rewind() on the reader object makes sure we're at the start of the feature list
# This way, we can execute this cell multiple times and get the same result 
wfreader.rewind()
# Now, read through each of the features, saving them as dictionaries into a list
feature = wfreader.next()
while feature:
    feature_list.append(feature)
    feature_count += 1
    # if we're loading a lot of features, print progress
    if (feature_count % 10000) == 0:
        print(f"Loaded {feature_count} features")
    # loaded the max we're allowed then break
    if feature_count >= MAX_FEATURE_LOAD:
        break
    feature = wfreader.next()
#
#    Print the number of items (features) we think we loaded
print(f"Loaded a total of {feature_count} features")
#
#    Just a validation check - did all the items we loaded get into the list?
print(f"Variable 'feature_list' contains {len(feature_list)} features")

Loaded 10000 features
Loaded 20000 features
Loaded 30000 features
Loaded 40000 features
Loaded 50000 features
Loaded 60000 features
Loaded 70000 features
Loaded 80000 features
Loaded 90000 features
Loaded 100000 features
Loaded 110000 features
Loaded 120000 features
Loaded 130000 features
Loaded a total of 135061 features
Variable 'feature_list' contains 135061 features


Now that I've imported the data into my environment, the first thing I need to do is calculate two notions of distance:
- the distance from Pueblo, CO to the **edge** of the fire
- the distance from Pueblo, CO to the **center** of the fire

The three methods below this cell respectively:
- ensure that the geometries in the wildfire data are expressed in terms of a consistent map projection
- calculates the distance from Pueblo, CO to the closest point in the fire area
- calculates the distance from Pueblo, CO to the centroid of the fire area

In [5]:
geodcalc = Geod(ellps='WGS84')  # ensures that coordinates in this notebook are
                                # expressed in the correct projection

#    Two constants for accessing the 'latlon' array in our CITY_LOCATIONS constant dict
LAT = 0
LON = 1

#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        try:
            lat,lon = to_epsg4326.transform(coord[0],coord[1])
            new_coord = lat,lon
            converted_ring.append(new_coord)
        # sometimes, coords in this data are not correctly formatted; instead
        # of being [lat, lon] they are dicts like {'a': [[lat, lon], [lat, lon]]}...
        # if this is the case; I'm skipping it. Should not materially affect
        # the distance calculation in this use case
        except:
            converted_ring.append(None)
    return converted_ring

#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        if point is None:
            continue
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point



#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        if point is None:
            continue
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    try:
        average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    except:
        average = float('inf')
    return average



Now I implement the functions defined above to calculate and store the two notions of distance.

In [5]:
# Source: https://wiki.openstreetmap.org/wiki/Pueblo,_Colorado#:~:text=Pueblo%20is%20a%20town%20in,°37′13.00″%20West.
place = {
    "city": "Pueblo, CO",
    "latlon": [38.2706, -104.6101]   
}

# store the distances
shortest_distances = []
average_distances = []

for wf_feature in tqdm(feature_list):
    
    # fires may be represented using two different kinds of geometry: ring or curve Ring
    # check that one of these holds, otherwise raise an error
    if 'rings' in wf_feature['geometry']:
        ring_data = wf_feature['geometry']['rings'][0]
    elif 'curveRings' in wf_feature['geometry']:
        ring_data = wf_feature['geometry']['curveRings'][0]
    else:
        raise Exception("HEY! No compatible geometry in this fire data!!!")
    #
    #     Compute using the shortest distance to any point on the perimeter
    #
    distance = shortest_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
    shortest_distances.append(distance)

    distance = average_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
    average_distances.append(distance)

100%|██████████| 135061/135061 [38:02<00:00, 59.17it/s]


Having computed the distances, I dump this back into the original fire wildfire data structure, then save to my [`intermediate_data`](../intermediate_data/) folder.

In [ ]:
for i, feature in tqdm(enumerate(feature_list)):
    # store both distance notions for each fire in the dataset
    feature["attributes"]["shortest_distance_to_pueblo"] = shortest_distances[i]
    feature["attributes"]["centroid_distance_to_pueblo"] = average_distances[i]

In [10]:
with open("./intermediate_data/fire_features_with_distances.json", "w") as file:
    json.dump(feature_list, file, indent = 4) # indent argument ensures nice formatting

Now I've calculated the distance from Pueblo, CO to each of the wildfires in my datasets. In the other notebooks in this analysis, I will:
- Pull Air Quality Index (AQI) data from the EPA describing how different particulate and gaseous contaminants affect air quality
    - [`getting_aqi_data.ipynb`](getting_aqi.data.ipynb)
- Use the distances computed here and the acreages included in this dataset to develop a custom metric of wildfire impact & compare it to AQI
    - [`combining_distance_and_aqi.ipynb`](combining_distance_and_aqi.ipynb)